In [2]:
from __future__ import annotations
import math
import pandas as pd

In [4]:
def heversine(lon1: float, lat1: float, lon2: float, lat2: float):
    R = 6371.0  #Radius of the Earth in km
    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)
    
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = R * c
    
    return distance

In [7]:
class Checkpoint:
    def __init__(self, lat: float, lon: float, distance: float, elevation: float, cloud_cover: int, wind_dir: float, wind_speed: float):
        self.lat = lat
        self.lon = lon
        self.distance = distance
        self.elevation = elevation
        self.cloud_cover = cloud_cover
        self.wind_dir = wind_dir
        self.wind_speed = wind_speed
    
    def __str__(self):
        return f"Lat: {self.lat} | Lon: {self.lon} | Elevation: {self.elevation} | Cloud Cover: {self.cloud_cover} | Wind Dir: {self.wind_dir} | Wind Speed: {self.wind_speed}"
    
    def __repr__(self):
        return f"Lat: {self.lat} | Lon: {self.lon} | Elevation: {self.elevation} | Cloud Cover: {self.cloud_cover} | Wind Dir: {self.wind_dir} | Wind Speed: {self.wind_speed}"

In [49]:
class Route:
    def __init__(self, name: str, checkpoints: list[Checkpoint]):
        self.name = name
        self.checkpoints = checkpoints
    def __str__(self):
        return f"{self.name}: {self.checkpoints}"
    
    def __repr__(self):
        return f"{self.name}: {self.checkpoints}"

In [50]:
import csv

def load_csv(name: str):
    path = f"data/generated/{name}.csv"
    checkpoints: list[Checkpoint] = []
    with open(path, newline='') as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        next(reader)
        next(reader)
        for row in reader:
            checkpoint = Checkpoint(float(row[0]), float(row[1]), float(row[2]), float(row[3]), int(row[4]), float(row[5]), float(row[6]))
            checkpoints.append(checkpoint)
    return Route(name, checkpoints)


In [63]:
def match_coords_to_checkpoint(route: Route, location: tuple[float,float]):
    closest_point = (0, math.inf);
    for (i, coord) in enumerate(route.checkpoints):
        new_distance = heversine(coord.lon, coord.lat, location[1], location[0])
        if new_distance < closest_point[1]:
            closest_point = (i,new_distance)
    
    return closest_point

In [64]:
route1 = load_csv("A. Independence to Topeka")
print(match_coords_to_checkpoint(route1, (38.9932976, -94.4785269)))
print(route1.checkpoints[402])

(402, 0.28692740242282605)
Lat: 38.993079 | Lon: -94.481835 | Elevation: 315.928799999942 | Cloud Cover: 92 | Wind Dir: 338.0 | Wind Speed: 2.0


In [73]:
import folium
import pandas as pd
def route_to_list(route: Route):
    coords: list[tuple[float, float]] = [] 
    for checkpoint in route.checkpoints:
        coords.append((checkpoint.lat, checkpoint.lon))
    return coords

def create_map(route: Route, random_coord: tuple[float, float]):
    coords = route_to_list(route)
    m = folium.Map()
    folium.Marker(random_coord, popup="random", draggable=True).add_to(m)
    for coord in coords[0::50]:
        folium.Marker(coord).add_to(m)
    folium.PolyLine(coords, weight=5, opacity=1).add_to(m)
    nearest_coord_index = match_coords_to_checkpoint(route, random_coord)[0]
    nearest_coord = (route.checkpoints[nearest_coord_index].lat, route.checkpoints[nearest_coord_index].lon)
    folium.PolyLine([random_coord, nearest_coord],color="#FF0000", weight=5, opacity=1,tooltip="Ok").add_to(m)
    df = pd.DataFrame(coords).rename(columns={0: 'lat', 1: 'lon'})
    print(df)
    sw = df[['lat', 'lon']].min().values.tolist()
    ne = df[['lat', 'lon']].max().values.tolist()
    m.fit_bounds([sw, ne])
    return m

m = create_map(route1, (38.9932976, -94.4785269))
m

            lat        lon
0     39.092185 -94.417077
1     39.092184 -94.417187
2     39.092197 -94.417811
3     39.092212 -94.418348
4     39.092216 -94.418486
...         ...        ...
2569  39.038138 -95.675183
2570  39.038337 -95.675887
2571  39.038268 -95.675943
2572  39.037843 -95.676128
2573  39.037758 -95.676164

[2574 rows x 2 columns]
